<a href="https://colab.research.google.com/github/Wannayli/Assignment-9/blob/main/Exercise_Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check the environment

In [ ]:
!java --version
!python --version

openjdk 11.0.18 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
Python 3.9.16


Installing Apache Spark (PySpark)

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 16.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=bc918938c72682e60ff54fc17cefb3f060d6b4652cc7ce943c1843f844187934
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


Initialize Apache Spark context

In [ ]:
# Import Apache Spark SQL
from pyspark.sql import SparkSession

# Create Spark Session/Context
# We are using local machine with all the CPU cores [*]
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Hello Pyspark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
# Check spark session
print(spark)

Collaborative Filtering Task

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row


In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=620c6182b8265dba031203df4e1a895e2e1057c6b9a3dd272febd954936dfc2d
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


import wget - to download the file

In [ ]:
import wget

In [ ]:
file = wget.download("https://raw.githubusercontent.com/apache/spark/master/data/mllib/als/sample_movielens_ratings.txt")

In [ ]:
lines = spark.read.text("/content/sample_movielens_ratings.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])


Change the maxIter and regparam up to you for exampIe

In [ ]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.1, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [ ]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.062598674054944


.show() - is to show the recommendations

In [ ]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    20|[{75, 3.9864678},...|
|    10|[{2, 3.8715858}, ...|
|     0|[{62, 5.2895336},...|
|     1|[{32, 5.1827693},...|
|    21|[{29, 5.027496}, ...|
|    11|[{52, 5.9091544},...|
|    12|[{90, 5.9330916},...|
|    22|[{88, 5.2587886},...|
|     2|[{93, 5.2126493},...|
|    13|[{93, 3.5651164},...|
|     3|[{51, 4.882038}, ...|
|    23|[{55, 5.1419315},...|
|     4|[{92, 5.056689}, ...|
|    24|[{30, 5.1313796},...|
|    14|[{29, 5.134136}, ...|
|     5|[{55, 4.5865602},...|
|    15|[{77, 3.8668158},...|
|    25|[{52, 4.1323156},...|
|    26|[{51, 5.8584294},...|
|     6|[{25, 4.426136}, ...|
+------+--------------------+
only showing top 20 rows



In [ ]:
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10).show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     20|[{17, 4.9537897},...|
|     40|[{2, 3.940611}, {...|
|     10|[{23, 4.1530485},...|
|     50|[{12, 3.9121869},...|
|     80|[{3, 4.100973}, {...|
|     70|[{4, 3.9002895}, ...|
|     60|[{25, 3.519311}, ...|
|     90|[{12, 5.9330916},...|
|     30|[{24, 5.1313796},...|
|      0|[{0, 3.4648867}, ...|
|     31|[{21, 3.9330428},...|
|     81|[{28, 5.036371}, ...|
|     91|[{12, 3.3515444},...|
|      1|[{11, 3.273325}, ...|
|     41|[{8, 4.952047}, {...|
|     61|[{18, 2.424725}, ...|
|     51|[{26, 5.8584294},...|
|     21|[{26, 2.96658}, {...|
|     11|[{2, 2.352095}, {...|
|     71|[{25, 3.5633023},...|
+-------+--------------------+
only showing top 20 rows



In [ ]:
# Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    26|[{51, 5.8584294},...|
|    19|[{51, 4.2897463},...|
|    29|[{46, 5.0582056},...|
+------+--------------------+



In [ ]:
# Generate top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10).show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     65|[{11, 2.4886363},...|
|     26|[{0, 3.0141833}, ...|
|     29|[{14, 5.134136}, ...|
+-------+--------------------+



**Conclusion**

To conclude which models performs better, according to the model with lower RMSE.

For example, you can try 

maxIter = 5 | regParam = 0.1 the RMSE = 1.9583525038890754

maxIter = 10 | regParam = 0.5 the RMSE = 1.26794427489228

maxIter = 20 | regParam = 1.0 the RMSE = 1.0239071747072548

The model with the better performance according to the RMSE is the model with 20 maxIter and 1.0 regParam.
